In [2]:
import pandas as pd
import glob
import feature_engineering
import numpy as np

In [3]:
def get_df_from_folder(foldername):
    df = pd.read_excel(glob.glob(foldername+'*')[1])

    for filename in glob.glob(foldername+'*')[1:]:
        df_temp = pd.read_excel(filename)
        df = pd.concat([df, df_temp])

    df.reset_index(drop=True, inplace=True)
    
    return df

def get_names_list(filepath):
    list_names = pd.read_csv(filepath, header=None)
    names = []
    # Add the female and male names to their respective lists
    for name in list_names[0]:
        names.append(name)
    return names

def get_gender(name, female_names_list, male_names_list):
    if name in female_names_list:
        gender = "female"
    elif name in male_names_list:
        gender = "male"
    else:
        gender = "unknown"
    return gender

In [4]:
# Text files with female and male names
fnames_filepath = './../Data/Names/female_names.txt'
mnames_filepath = './../Data/Names/male_names.txt'

# Annotated data folder
foldername = './../Data/annotated_xlsx/'
len(glob.glob(foldername + '*'))

150

In [5]:
df = get_df_from_folder(foldername)

### Change "Examinded by" for annotated as neutral speech to annotated as not speech
for index, row in df.iterrows():
    #print(row)
    if "Examined by"  in row['text']:
        df.at[index, 'label'] = 5

In [6]:
print('total:', df.shape[0])
print('Powerless:', df[df['label']==1].shape[0])
print('Neutral:', df[df['label']==2].shape[0])
print('Powerful', df[df['label']==3].shape[0])
print('Mixed:', df[df['label']==4].shape[0])
print('5:', df[df['label']==5].shape[0])

total: 2242
Powerless: 439
Neutral: 1361
Powerful 200
Mixed: 66
5: 176


In [7]:
# Keep only powerless, neutral and powerful sentences
df = df[df['label'] <= 3]
df.reset_index(drop=True, inplace=True)

# Add features
#df = feature_engineering.add_feature_columns(df)

In [8]:
df.shape[0]

2000

In [8]:
fname_list = get_names_list(fnames_filepath)
mname_list = get_names_list(mnames_filepath)

In [9]:
df['gender'] = df.apply(lambda row: get_gender(row['first_name'], fname_list, mname_list), axis=1)

In [10]:
df.to_csv('./../Data/Old_Bailey_speech_features_v1.csv')